# Variational Quantum Classifier (VQC) Best Parameters

This notebook will explore the the options for the best parameters to use for the VQC on the Pima Indians Dataset

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

### Imports

In [2]:
# imports
import os
import time

from sklearn.utils import shuffle
from imblearn.over_sampling import SMOTE

# import data class
from utilities.dataset_utils import DiabetesData

# import metrics for evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

# qiskit imports
# circuit transpiler
from qiskit import transpile

# algorithm
from qiskit_machine_learning.algorithms.classifiers import VQC

# feature map
from qiskit.circuit.library import z_feature_map
from qiskit.circuit.library import zz_feature_map

# ansatz
from qiskit.circuit.library import real_amplitudes
from qiskit.circuit.library import efficient_su2

# optimizer
from qiskit_machine_learning.optimizers import COBYLA

# simulator
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import SamplerV2 as Sampler

In [3]:
# path to diabetes.csv
path = os.path.join(os.getcwd(), '..', '..', 'utilities', 'diabetes.csv')
# load dataset class
dataset = DiabetesData(path)

In [4]:
# setup backend simulator
backend = AerSimulator()
backend.set_options(max_parallel_threads=os.cpu_count(), method='automatic')

# sampler
sampler = Sampler.from_backend(backend)

In [5]:
# define a list for different entanglement patters
entanglement_patterns = [
    'full',
    'linear'
]

In [6]:
# define list for different ansatze as well as reps
ansatze = [
    'su2',
    'real_amp'
]

def build_ansatz(name, reps, entanglement):
    if name=="su2":
        return transpile(efficient_su2(num_qubits=dataset.get_num_features(), reps=reps, entanglement=entanglement), backend=backend, optimization_level=3)
    else:
        return transpile(real_amplitudes(num_qubits=dataset.get_num_features(), reps=reps, entanglement=entanglement), backend=backend, optimization_level=3)

In [7]:
# list of different feature maps
feature_maps = [
    'z',
    'zz'
]

def build_feature_map(name, reps, entanglement):
    if name=='z':
        return transpile(z_feature_map(feature_dimension=dataset.get_num_features(), reps=reps, entanglement=entanglement), backend=backend, optimization_level=3)
    else:
        return transpile(zz_feature_map(feature_dimension=dataset.get_num_features(), reps=reps, entanglement=entanglement), backend=backend, optimization_level=3)

### VQC Building

Below is a function that will be iteratively called with multiple different parameters for training the VQC, to find the most optimal combination

In [8]:
def build_vqc(ansatz, feature_map, optimizer):
    vqc =  VQC(
        sampler=sampler,
        feature_map=feature_map,
        ansatz=ansatz,
        optimizer=optimizer
    )
    return vqc

Below function find the best combination of parameters for the VQC, as well as different entanglement patterns

In [9]:
def combo_builder(ansatze: list, feature_maps: list, optimizer, entanglement_patterns: list, X_train, y_train, X_test, y_test):
    # list to store results
    results = []
    # combo counter
    combo = 0
    # rep ranges feature map
    for i in range(1, 4):
        feature_map_reps = i+1
        # rep ranges ansatz
        for j in range(1, 4):
            ansatz_reps = j+1
            # itereate over different ansatze
            for ansatz_name in ansatze:
                # iterate over different feature maps
                for feature_map_name in feature_maps:
                    for fm_pattern in entanglement_patterns:
                        for a_pattern in entanglement_patterns:
                            # build ansatz with given params
                            ansatz = build_ansatz(ansatz_name, ansatz_reps, a_pattern)
                            # build feature map with given params
                            feature_map = build_feature_map(feature_map_name, feature_map_reps, fm_pattern)
                            combo += 1
                            
                            # print combo information
                            print(f"=====COMBINATION: {combo}=====\nAnsatz: {ansatz.name}\nAnsatz Reps: {ansatz_reps}\nFeature Map: {feature_map.name}\nFeature Map Reps: {feature_map_reps}\nOptimizer: {type(optimizer).__name__}\n")
                            
                            # try parameter combo
                            test_vqc = build_vqc(ansatz, feature_map, optimizer)

                            # time how long it takes to train
                            start = time.time()

                            # fit the model
                            test_vqc = test_vqc.fit(X_train, y_train)

                            end = time.time()
                            elapsed = end - start
                            
                            # sleep to avoid crashing
                            time.sleep(10)
                            
                            train_score = test_vqc.score(X_train, y_train)
                            test_score = test_vqc.score(X_test, y_test)
                            
                            # sleep to avoid crashing
                            time.sleep(10)
                            
                            # define scores for results
                            y_pred = test_vqc.predict(X_test)
                            accuracy = accuracy_score(y_test, y_pred)
                            p_r_f1_s = precision_recall_fscore_support(y_test, y_pred)
                            print(f"ACCURACY: {accuracy}")
                            print(f"P R F1 S: {p_r_f1_s}")

                            data = {
                                "combo_num": combo,
                                "ansatz": ansatz.name,
                                "ansatz_reps": ansatz_reps,
                                "feature_map": feature_map.name,
                                "fm_reps": feature_map_reps,
                                "optmizer": type(optimizer).__name__,
                                "train_time": elapsed,
                                "train_score": train_score,
                                "test_score": test_score,
                                "accuracy": accuracy,
                                "p_r_f1_s": p_r_f1_s
                            }
                        
                            results.append(data)
                            print("=====COMBINATION COMPLETED=====")
                            
                            # sleep to avoid crashing
                            time.sleep(10)
    return results

### Model Fit for Best Params

Ranged preprocessing used between (-$\pi$, $\pi$) to allow full exploration of the bloch sphere.

In [10]:
# get training and testing data
X_train, X_test, y_train, y_test = dataset.preprocess_data_ranged()

In [11]:
# apply smote to help with class imbalance
# in testing it proved to improve model performance on unseen data
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)
X_train_balanced, y_train_balanced = shuffle(X_train_balanced, y_train_balanced, random_state=42)

In [12]:
# split up feature maps used, as this function is very computationally intensive
comb_z = combo_builder(ansatze, feature_maps[:1], COBYLA(maxiter=500), entanglement_patterns, X_train_balanced, y_train_balanced, X_test, y_test)
print(comb_z)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


=====COMBINATION: 1=====
Ansatz: EfficientSU2
Ansatz Reps: 2
Feature Map: ZFeatureMap
Feature Map Reps: 2
Optimizer: COBYLA

ACCURACY: 0.5757575757575758
P R F1 S: (array([0.72807018, 0.42735043]), array([0.55333333, 0.61728395]), array([0.62878788, 0.50505051]), array([150,  81]))
=====COMBINATION COMPLETED=====


No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


=====COMBINATION: 2=====
Ansatz: EfficientSU2
Ansatz Reps: 2
Feature Map: ZFeatureMap
Feature Map Reps: 2
Optimizer: COBYLA

ACCURACY: 0.5930735930735931
P R F1 S: (array([0.7       , 0.42857143]), array([0.65333333, 0.48148148]), array([0.67586207, 0.45348837]), array([150,  81]))
=====COMBINATION COMPLETED=====


No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


...

=====COMBINATION: 70=====
Ansatz: RealAmplitudes
Ansatz Reps: 4
Feature Map: ZFeatureMap
Feature Map Reps: 4
Optimizer: COBYLA

ACCURACY: 0.5497835497835498
P R F1 S: (array([0.70535714, 0.40336134]), array([0.52666667, 0.59259259]), array([0.60305344, 0.48      ]), array([150,  81]))
=====COMBINATION COMPLETED=====


No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


=====COMBINATION: 71=====
Ansatz: RealAmplitudes
Ansatz Reps: 4
Feature Map: ZFeatureMap
Feature Map Reps: 4
Optimizer: COBYLA

ACCURACY: 0.5670995670995671
P R F1 S: (array([0.66025641, 0.37333333]), array([0.68666667, 0.34567901]), array([0.67320261, 0.35897436]), array([150,  81]))
=====COMBINATION COMPLETED=====


No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


=====COMBINATION: 72=====
Ansatz: RealAmplitudes
Ansatz Reps: 4
Feature Map: ZFeatureMap
Feature Map Reps: 4
Optimizer: COBYLA

ACCURACY: 0.5411255411255411
P R F1 S: (array([0.67741935, 0.38317757]), array([0.56      , 0.50617284]), array([0.61313869, 0.43617021]), array([150,  81]))
=====COMBINATION COMPLETED=====
[{'combo_num': 1, 'ansatz': 'EfficientSU2', 'ansatz_reps': 2, 'feature_map': 'ZFeatureMap', 'fm_reps': 2, 'optmizer': 'COBYLA', 'train_time': 1972.9104430675507, 'train_score': 0.5771428571428572, 'test_score': 0.5887445887445888, 'accuracy': 0.5757575757575758, 'p_r_f1_s': (array([0.72807018, 0.42735043]), array([0.55333333, 0.61728395]), array([0.62878788, 0.50505051]), array([150,  81]))}, {'combo_num': 2, 'ansatz': 'EfficientSU2', 'ansatz_reps': 2, 'feature_map': 'ZFeatureMap', 'fm_reps': 2, 'optmizer': 'COBYLA', 'train_time': 1887.9535567760468, 'train_score': 0.6485714285714286, 'test_score': 0.5930735930735931, 'accuracy': 0.5930735930735931, 'p_r_f1_s': (array([0.7 

In [13]:
comb_zz = combo_builder(ansatze, feature_maps[1:], COBYLA(maxiter=500), entanglement_patterns, X_train_balanced, y_train_balanced, X_test, y_test)
print(comb_zz)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


=====COMBINATION: 1=====
Ansatz: EfficientSU2
Ansatz Reps: 2
Feature Map: ZZFeatureMap
Feature Map Reps: 2
Optimizer: COBYLA

ACCURACY: 0.5064935064935064
P R F1 S: (array([0.66071429, 0.36134454]), array([0.49333333, 0.5308642 ]), array([0.5648855, 0.43     ]), array([150,  81]))
=====COMBINATION COMPLETED=====


No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


=====COMBINATION: 2=====
Ansatz: EfficientSU2
Ansatz Reps: 2
Feature Map: ZZFeatureMap
Feature Map Reps: 2
Optimizer: COBYLA

ACCURACY: 0.5151515151515151
P R F1 S: (array([0.6557377 , 0.35779817]), array([0.53333333, 0.48148148]), array([0.58823529, 0.41052632]), array([150,  81]))
=====COMBINATION COMPLETED=====


No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


...

=====COMBINATION: 66=====
Ansatz: EfficientSU2
Ansatz Reps: 4
Feature Map: ZZFeatureMap
Feature Map Reps: 4
Optimizer: COBYLA

ACCURACY: 0.5021645021645021
P R F1 S: (array([0.65765766, 0.35833333]), array([0.48666667, 0.5308642 ]), array([0.55938697, 0.4278607 ]), array([150,  81]))
=====COMBINATION COMPLETED=====


No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


=====COMBINATION: 67=====
Ansatz: EfficientSU2
Ansatz Reps: 4
Feature Map: ZZFeatureMap
Feature Map Reps: 4
Optimizer: COBYLA



capi_return is NULL
Call-back cb_calcfc_in__cobyla__user__routines failed.


KeyboardInterrupt: 